# Aula 7: Web Scrapping

Web scrapping ou raspagem de telas, aplicações e páginas são processos executados para coleta de dados das quais podem serem realizadas diretamente nas APIs que servem as aplicações, nas páginas estáticas ou simulando consições específicas do navegador.

Este tipo de coleta pode envolver grande complexidade se for adotado a coleta pelo navegador e diretamente pela página estática. Portanto procure identificar, com ferramentas de desenvolvedor, qual é a API utilizada e faça as chamadas diretamente delas.

Caso não seja possível a coleta direta das APIs, parta para a análise de página estática e, somente por último, analise a coleta pelo navegador

## Biblioteca Requests

A biblioteca requests permite acesso direto tanto a páginas estáticas quando a métodos das APIs Restful, sendo assim a primeira opção de coleta de dados dado sua simplicidade.

In [ ]:
import requests

Para exemplificar, vamos acessar a API do INPE para coleta de daods de metereologia. A API retorna os dados no formato XML.

In [ ]:
r = requests.get("http://servicos.cptec.inpe.br/XML/estacao/SBMT/condicoesAtuais.xml")

In [ ]:
r

<Response [200]>

In [ ]:
resultado = r.text
resultado

"<?xml version='1.0' encoding='ISO-8859-1'?><metar><codigo>SBMT</codigo><atualizacao>11/07/2023 08:00:00</atualizacao><pressao>1019</pressao><temperatura>18</temperatura><tempo>ps</tempo><tempo_desc>PredomÃ\xadnio de Sol</tempo_desc><umidade>100</umidade><vento_dir>130</vento_dir><vento_int>4</vento_int><visibilidade>>10000</visibilidade></metar>"

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
xml = ET.fromstring(resultado)

for table in xml.iter():
    for child in table:
        print(child.tag, child.text)

nome Rio de Janeiro
uf RJ
atualizacao 16-07-2023
manha None
tarde None
noite None
dia 16-07-2023 12h Z
agitacao Fraco
altura 1.6
direcao ESE
vento 3.8
vento_dir NNE
dia 16-07-2023 18h Z
agitacao Fraco
altura 1.4
direcao ESE
vento 3.5
vento_dir SE
dia 16-07-2023 21h Z
agitacao Fraco
altura 1.4
direcao ESE
vento 4.5
vento_dir E


In [ ]:
r = requests.get("http://servicos.cptec.inpe.br/XML/cidade/241/dia/0/ondas.xml")
r

<Response [200]>

In [ ]:
resultado = r.text
resultado

"<?xml version='1.0' encoding='ISO-8859-1'?><cidade><nome>Rio de Janeiro</nome><uf>RJ</uf><atualizacao>16-07-2023</atualizacao><manha><dia>16-07-2023 12h Z</dia><agitacao>Fraco</agitacao><altura>1.6</altura><direcao>ESE</direcao><vento>3.8</vento><vento_dir>NNE</vento_dir></manha><tarde><dia>16-07-2023 18h Z</dia><agitacao>Fraco</agitacao><altura>1.4</altura><direcao>ESE</direcao><vento>3.5</vento><vento_dir>SE</vento_dir></tarde><noite><dia>16-07-2023 21h Z</dia><agitacao>Fraco</agitacao><altura>1.4</altura><direcao>ESE</direcao><vento>4.5</vento><vento_dir>E</vento_dir></noite></cidade>"

In [ ]:
xml = ET.fromstring(resultado)

for table in xml.iter():
    for child in table:
        print(child.tag, child.text)

nome Rio de Janeiro
uf RJ
atualizacao 16-07-2023
manha None
tarde None
noite None
dia 16-07-2023 12h Z
agitacao Fraco
altura 1.6
direcao ESE
vento 3.8
vento_dir NNE
dia 16-07-2023 18h Z
agitacao Fraco
altura 1.4
direcao ESE
vento 3.5
vento_dir SE
dia 16-07-2023 21h Z
agitacao Fraco
altura 1.4
direcao ESE
vento 4.5
vento_dir E


## Parseando e simplificando dados de apresentação com a biblioteca Beautiful Soup

O Beautiful Soup nos traz maior facilidade de análise de páginas estáticas ao parsear o conteúdo HTML e permitir a busca de seus elementos.

Geralmente baixamos a página estática pelo Requests e analisamos o conteúdo com o Beaultiful Soup.

Vamos obter os dados de cotação de dólar a partir de uma faixa de datas, conforme abaixo.

Ao analisar a página de consulta, percebemos que ela envia uma requisição do tipo post para o mesmo endpoint da página estática, com isso extraímos os parâmetros e incluímos na requisição.

In [ ]:
parametros = {"RadOpcao": 1,"DATAINI": "25/09/2021", "DATAFIM": "24/10/2021", "ChkMoeda": 61}

In [ ]:
r = requests.post("https://ptax.bcb.gov.br/ptax_internet/consultaBoletim.do?method=consultarBoletim",params=parametros)
r

<Response [200]>

O resultado é uma outra página renderizada, da qual possui diversos componentes HTML. Nosso objetivo é extrair somente os dados de tabela e obter seus valores.

In [ ]:
resultado = r.text
resultado

'\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html>\r\n    <head>\r\n        <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1">\r\n        <link rel="stylesheet" type="text/css" href="/ptax_internet/ncss/style.css">\r\n        <title></title>\r\n    </head>\r\n    <body>\r\n\r\n        <div style="text-align: center">\r\n            <br>\r\n            Cotações de Fechamento Ptax<sup>4/</sup> do DOLAR DOS EUA, Código da Moeda: 220, Símbolo da Moeda: USD, Tipo da Moeda: A, período de 25/09/2021 a 24/10/2021.\r\n            <br>\r\n            <br>\r\n\r\n            Clique para obter a tabela completa (\r\n            <a href="/ptax_internet/consultaBoletim.do?method=gerarCSVFechamentoMoedaNoPeriodo&ChkMoeda=61&DATAINI=25/09/2021&DATAFIM=24/10/2021">\r\n                <img src="/img/transferirA.GIF" border="0" longdesc="img">&nbsp; CSV - 2 KB\r\n            </a>)\r\n            <br>\r\n     

Nosso primeiro passo é parsear a página resultante com o Beautiful Soup.

In [ ]:
from bs4 import BeautifulSoup

Ao analisar a página resultante, notamos que a tabela possui 2 classes de estilo de interesse, responsável por variar a cor das linhas, e que nos ajudou a filtrar os componentes que precisams.

In [ ]:
bs = BeautifulSoup(resultado, 'html.parser')

items = bs.find_all('tr',{"class": ["fundoPadraoBClaro2","fundoPadraoBClaro3"]})
items

[<tr class="fundoPadraoBClaro2">
 <td>27/09/2021</td>
 <td>A</td>
 <td>5,3472</td>
 <td>5,3478</td>
 </tr>,
 <tr class="fundoPadraoBClaro3">
 <td>28/09/2021</td>
 <td>A</td>
 <td>5,4200</td>
 <td>5,4206</td>
 </tr>,
 <tr class="fundoPadraoBClaro2">
 <td>29/09/2021</td>
 <td>A</td>
 <td>5,4167</td>
 <td>5,4173</td>
 </tr>,
 <tr class="fundoPadraoBClaro3">
 <td>30/09/2021</td>
 <td>A</td>
 <td>5,4388</td>
 <td>5,4394</td>
 </tr>,
 <tr class="fundoPadraoBClaro2">
 <td>01/10/2021</td>
 <td>A</td>
 <td>5,3905</td>
 <td>5,3911</td>
 </tr>,
 <tr class="fundoPadraoBClaro3">
 <td>04/10/2021</td>
 <td>A</td>
 <td>5,4198</td>
 <td>5,4204</td>
 </tr>,
 <tr class="fundoPadraoBClaro2">
 <td>05/10/2021</td>
 <td>A</td>
 <td>5,4605</td>
 <td>5,4611</td>
 </tr>,
 <tr class="fundoPadraoBClaro3">
 <td>06/10/2021</td>
 <td>A</td>
 <td>5,5091</td>
 <td>5,5097</td>
 </tr>,
 <tr class="fundoPadraoBClaro2">
 <td>07/10/2021</td>
 <td>A</td>
 <td>5,5134</td>
 <td>5,5140</td>
 </tr>,
 <tr class="fundoPadraoBClar

In [ ]:
items[0]

<tr class="fundoPadraoBClaro2">
<td>27/09/2021</td>
<td>A</td>
<td>5,3472</td>
<td>5,3478</td>
</tr>

In [ ]:
items[1]

<tr class="fundoPadraoBClaro3">
<td>28/09/2021</td>
<td>A</td>
<td>5,4200</td>
<td>5,4206</td>
</tr>

Depois de analisar cada linha, analisaremos cada coluna e assim compor nosso dataframe com a evolução da cotação do dólar.

In [ ]:
sub_item = items[0].findChildren('td')
sub_item

[<td>27/09/2021</td>, <td>A</td>, <td>5,3472</td>, <td>5,3478</td>]

A tabela tem uma ordem para cada coluna, sendo a data, o tipo, o valor de compra e venda.

In [ ]:
for i in items:
  sub_items = i.findChildren("td")
  for si in sub_items:
    print(si.text)
  break

27/09/2021
A
5,3472
5,3478


In [ ]:
data, tipo, compra, venda = [], [], [], []

In [ ]:
campo = "data"
for i in items:
  sub_items = i.findChildren("td")
  for si in sub_items:
    if campo == "data":
      data.append(si.text)
      campo = "tipo"
    elif campo == "tipo":
      campo = "compra"
      tipo.append(si.text)
    elif campo == "compra":
      campo = "venda"
      compra.append(si.text)
    else:
      campo = "data"
      venda.append(si.text)

In [ ]:
data

['27/09/2021',
 '28/09/2021',
 '29/09/2021',
 '30/09/2021',
 '01/10/2021',
 '04/10/2021',
 '05/10/2021',
 '06/10/2021',
 '07/10/2021',
 '08/10/2021',
 '11/10/2021',
 '13/10/2021',
 '14/10/2021',
 '15/10/2021',
 '18/10/2021',
 '19/10/2021',
 '20/10/2021',
 '21/10/2021',
 '22/10/2021']

In [ ]:
compra

['5,3472',
 '5,4200',
 '5,4167',
 '5,4388',
 '5,3905',
 '5,4198',
 '5,4605',
 '5,5091',
 '5,5134',
 '5,5078',
 '5,5155',
 '5,5464',
 '5,4982',
 '5,4504',
 '5,5187',
 '5,5515',
 '5,5565',
 '5,6417',
 '5,7111']

In [ ]:
import pandas as pd

In [ ]:


df = pd.DataFrame({"Data": data, "Tipo": tipo, "Compra": compra, "Venda": venda})
df

,Data,Tipo,Compra,Venda
0,27/09/2021,A,"5,3472","5,3478"
1,28/09/2021,A,"5,4200","5,4206"
2,29/09/2021,A,"5,4167","5,4173"
3,30/09/2021,A,"5,4388","5,4394"
4,01/10/2021,A,"5,3905","5,3911"
5,04/10/2021,A,"5,4198","5,4204"
6,05/10/2021,A,"5,4605","5,4611"
7,06/10/2021,A,"5,5091","5,5097"
8,07/10/2021,A,"5,5134","5,5140"
9,08/10/2021,A,"5,5078","5,5084"


## Utilizando Selenium para coleta de dados

O Selenium é uma biblioteca disponível em várias linguagens de programação cujo o principal objetivo foi de automatizar testes de interface. Utilizando essa capacidade de interação também podemos utilizar para coletar dados de sites onde a manipulação do navegador torna o processo ou obrigatório ou mais fácil, com o filtro por XPath, ausente no Beautiful Soup.

A instalação do Colab requer os passos a seguir:

In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

Executing: /tmp/apt-key-gpghome.X2XwWdQyXL/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.8cBOvmy8xi/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ROdPmkGvIK/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!apt-get install chromium chromium-driver
!pip install selenium

Vamos testar se deu tudo certo com as instalações. Se recebermos o retorno "Example Domain" é que estamos prontos com o Selenium.

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

url = "http://example.com/"
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
driver = webdriver.Chrome(options=options)
driver.get(url)
print(driver.title)
driver.quit()

Example Domain


Somente no Colab, precisamos configurar de forma especial o driver (navegador) utilizado.

In [8]:
from selenium.webdriver.common.by import By
import time

Toda vez que a inicialização do Selenium demorar muito, iremos fechar o driver e abrir novamente.

In [ ]:
driver.close()

In [85]:
driver = webdriver.Chrome(options=options)

Vamos coletar as notícias do portal do governo do CARF, que é o Conselho Administrativo de Recursos Fiscais.

Esse portal reúne diversas informações sobre recursos em casos de processos tributários.

In [87]:
url_root = 'http://idg.carf.fazenda.gov.br/noticias/ultimas-noticias'

driver = webdriver.Chrome(options=options)
driver.get(url_root)
time.sleep(5)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="28cf22f80d1620c133005ee5a2574bcd")>

Para ter certeza de que estamos coletando o que precisamos, vamos tirar um screenshot.

In [88]:
driver.save_screenshot("pagina_1.png")

True

Agora vamos usar o lcoalizador XPATH (não o XPATH completo) para obter o elemento da página.
Conseguimos obter o localizador diretamente no Chrome ao clicar no elemento, depois inspecionar, depois com o botão direito "Copiar">"Copiar XPATH".

In [89]:
item_1 = driver.find_element(By.XPATH,'//*[@id="content-core"]/div[1]/div/h2/a')
item_1.text

'Plataforma de pesquisa de Acórdãos e Resoluções do CARF – VER, completa dois anos com mais de meio milhão de documentos'

In [94]:
link_1 = item_1.get_attribute("href")
link_1

'http://idg.carf.fazenda.gov.br/noticias/2022-1/plataforma-de-pesquisa-de-acordaos-e-resolucoes-do-carf-2013-ver-completa-dois-anos-com-meio-mais-de-meio-milhao-de-documentos'

In [96]:
item_2 = driver.find_element(By.XPATH,'//*[@id="content-core"]/div[2]/div/h2/a')
item_2.text

'Estão suspensas todas as sessões de julgamento de Turmas Ordinárias, Extraordinárias e da Câmara Superior de Recursos Fiscais da semana de 22 a 25 de maio'

In [97]:
link_2 = item_2.get_attribute("href")
link_2

'http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recursos-fiscais-da-semana-de-23-a-25-de-maio'

Ao inspecionar a página, verificamos 30 entradas do elemento "div". Portanto podemos fazer um laço para obter cada uma das notícias desta página.

In [100]:
for idx in range(1,31):
  item = driver.find_element(By.XPATH,'//*[@id="content-core"]/div[' + str(idx) + ']/div/h2/a')
  link = item.get_attribute("href")
  print(item.text)
  print(link)

Plataforma de pesquisa de Acórdãos e Resoluções do CARF – VER, completa dois anos com mais de meio milhão de documentos
http://idg.carf.fazenda.gov.br/noticias/2022-1/plataforma-de-pesquisa-de-acordaos-e-resolucoes-do-carf-2013-ver-completa-dois-anos-com-meio-mais-de-meio-milhao-de-documentos
Estão suspensas todas as sessões de julgamento de Turmas Ordinárias, Extraordinárias e da Câmara Superior de Recursos Fiscais da semana de 22 a 25 de maio
http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recursos-fiscais-da-semana-de-23-a-25-de-maio
Estão suspensas todas as sessões de julgamento de Turmas Ordinárias, Extraordinárias e da Câmara Superior de Recursos Fiscais da semana de 16 a 18 de maio
http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recursos-fiscais-da-semana-de-16-a-18-de

Verificamos também que há 12 páginas de notícias disponíveis.
Podemos juntar os 2 laços em um mesmo para percorrer cada página.

Para não tornar lento o processamento, vamos percorrer 3 páginas.

Note que cada sistema possui um incrementador próprio. Nesse o paginador é o número de notícias, sendo um múltiplo de 30. Começamos com 0 na primeira página, 30 na segunda (+30), 60 na terceira (+30) e assim por diante.

In [101]:
paginas = 3

for pagina in range(1, paginas+1):
  print("Página " + str(pagina))
  num = 0
  url_root = 'http://idg.carf.fazenda.gov.br/noticias/ultimas-noticias?b_start:int=' + str(num)

  driver = webdriver.Chrome(options=options)
  driver.get(url_root)
  time.sleep(5)

  for idx in range(1,31):
    item = driver.find_element(By.XPATH,'//*[@id="content-core"]/div[' + str(idx) + ']/div/h2/a')
    link = item_2.get_attribute("href")
    print(item.text)
    print(link)


  num =+ 30

Página 1
Plataforma de pesquisa de Acórdãos e Resoluções do CARF – VER, completa dois anos com mais de meio milhão de documentos
http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recursos-fiscais-da-semana-de-23-a-25-de-maio
Estão suspensas todas as sessões de julgamento de Turmas Ordinárias, Extraordinárias e da Câmara Superior de Recursos Fiscais da semana de 22 a 25 de maio
http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recursos-fiscais-da-semana-de-23-a-25-de-maio
Estão suspensas todas as sessões de julgamento de Turmas Ordinárias, Extraordinárias e da Câmara Superior de Recursos Fiscais da semana de 16 a 18 de maio
http://idg.carf.fazenda.gov.br/noticias/2022-1/estao-suspensas-todas-as-sessoes-de-julgamento-de-turmas-ordinarias-extraordinarias-e-da-camara-superior-de-recu